In [541]:
'''
Converts output of API calls to dataframe. Index is created from variables, rows each represent a block group. 
'''

def json_list_to_dataframe(json_list):

    output_dictionary = {} 
    
    index_list = json_list[0][0]
    
    for index in index_list: 
        output_dictionary[index] = []

    for json in json_list: 
        values = json[1]

        for i, value in enumerate(values): 
            output_dictionary[index_list[i]].append(value)

    api_dataframe = pd.DataFrame.from_dict(output_dictionary)
    api_dataframe = api_dataframe.rename(columns = code_label_dictionary)
    
    api_dataframe['FIPS'] = api_dataframe['state'] + api_dataframe['county'] + api_dataframe['tract'] + api_dataframe['block group']
    
    return api_dataframe 

In [542]:
'''
Change columns to integers if value is numeric. 
'''
def change_columns_to_integers(dataframe): 
    
    for column in dataframe.columns: 

        try: 
            dataframe[column] = dataframe[column].astype(int)
        except: 
            pass
            
    return dataframe 

In [543]:
'''
Create column categories 
'''
def categorize_columns(api_dataframe): 
    
    column_category = {} 

    for column in api_dataframe.columns: 
        
        try: 
            if api_dataframe[column].dtype == int: 
                print(column)
                x = input() 
                if x != '': 
                    try: 
                        column_category[x].append(columm)
                    except: 
                        column_category[x] = [column]
        except: 
            print(f'No dtype: {column}')
                
    return column_category 

In [544]:
'''
Read and write text files to dictionary
'''
def dictionary_to_txt(txt_file, column_category_dict): 
    
    with open(txt_file, 'w') as convert_file: 
        convert_file.write(json.dumps(column_category_dict))
        
    return txt_file

def txt_to_dict(txt_file): 
    
    with open(txt_file) as read_file: 
        for line in read_file: 
            column_category_dict = json.loads(line)
        
    return column_category_dict

In [545]:
'''
Create dictionary to categorize columns 
'''
def create_column_category_dict(api_dataframe): 
    
    column_category_dict = {} 

    # Iterate through columns 
    for column in api_dataframe.columns: 
        if api_dataframe[column].dtype == int: 
            print(column)
            x = input() 

            if x != '': 
                column_category_dict[column] = x
                
    return column_category_dict

In [546]:
'''
Create dictionary to categorize columns 
'''
def create_column_category_dict(api_dataframe):
    
    # Create dictionary 
    column_category_dict = {} 
    
    # Iterate through columns 
    for column in api_dataframe.columns:  
        try: 
            if api_dataframe[column].dtype == int: 
                print(column)
                x = input() 

                if x != '': 
                    column_category_dict[column] = x
        except: 
            print(f'Passed: {column}')
                
    return column_category_dict

In [547]:
'''
Apply the dictionary to cluster columns 
'''
def drop_columns(column_category_dict, api_dataframe): 
    
    for column, category in column_category_dict.items(): 
        if category == 'DROP': 
            api_dataframe.drop(columns = column, inplace = True)
            print(f'Dropped: {column}')
            
    try: 
        api_dataframe.drop(columns = 'NAME', inplace = True)
    except: 
        pass
            
    return api_dataframe

def cluster_columns(column_category_dict, api_dataframe): 
    
    # Initialize new columns 
    for column, category in column_category_dict.items():
        if category != 'DROP': 
            api_dataframe[category] = 0
    
    # Add values into cluster columns and drop old columns 
    for column, category in column_category_dict.items():
        if category != 'DROP': 
            api_dataframe[category] += api_dataframe[column]
    
    # Drop remaining columns
    for column, category in column_category_dict.items(): 
        try: 
            api_dataframe.drop(columns = column, inplace = True)
        except: 
            pass

    return api_dataframe

In [559]:
'''
Calculate percentage columns. 
'''
def calculate_percentage_columns(dataframe):
    
    # Iterate through columns 
    for column in dataframe.columns:
        
        # If it is in the dictionary
        if column not in ['FIPS'] and api_dataframe[column].dtype == int: 
            # Identify the column's position  
            column_position = dataframe.columns.get_loc(column) + 1
            # Calculate its value
            column_value       = int(dataframe[column])
            percentage_formula = (column_value/dataframe['TOTAL'])*100
            # Insert the new column 
            dataframe.insert(column_position, f'% {column}', round(percentage_formula, 2))
            
    return dataframe 

In [549]:
'''
Master Function 
'''

def create_api_dataframe(json_list, column_category_dict): 

    api_dataframe        = json_list_to_dataframe(json_list)
    api_dataframe        = change_columns_to_integers(api_dataframe)
    
    if column_category_dict == {}: 
        column_category_dict = create_column_category_dict(api_dataframe)

    api_dataframe        = change_columns_to_integers(api_dataframe)
    
    api_dataframe        = drop_columns(column_category_dict, api_dataframe)
    api_dataframe        = cluster_columns(column_category_dict, api_dataframe)
    api_dataframe        = calculate_percentage_columns(api_dataframe)
    
    return api_dataframe